In [4]:
import sys
import os

# Hugging Face Transformers and PyTorch
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import TensorDataset, DataLoader


# Sklearn for Model Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import numpy as np


In [5]:
utils_path = r'C:\Users\aless\Desktop\final project 2.1\Final-Project---Luis-Augustin-Ale\utils'
sys.path.append(os.path.abspath(utils_path))

In [6]:
from custom_class_final_model import CustomRobertaModel


In [7]:
# Load the pre-trained model and tokenizer from Hugging Face
model_name = "AleOfDurin/final_retrained_model"
# Load the model and tokenizer from the Hugging Face Hub
model_custom = CustomRobertaModel.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

c:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aless\.cache\huggingface\hub\models--AleOfDurin--final_retrained_model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaForSequenceClassification were not initialized from the mo

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/506k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [8]:
# Move the model to the GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_custom.to(device)
model_custom.eval()  # Set the model to evaluation mode

CustomRobertaModel(
  (roberta): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
              

In [9]:
# Load your validation data (tokenized input_ids and attention_masks)
val_encodings, val_labels = torch.load(r'C:\Users\aless\Desktop\final project 2.1\val_encodings.pt')


C:\Users\aless\AppData\Local\Temp\ipykernel_199276\2492013525.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_encodings, val_labels = torch.load(r'C:\Users\aless\Des

In [10]:
# test tokens
texts = ["Your sample text here", "Another sample text"]  # Replace with your dataset's text

# Tokenize the texts
tokenized_inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Move tokenized inputs to the device
input_ids = tokenized_inputs['input_ids'].to(device)
attention_mask = tokenized_inputs['attention_mask'].to(device)

# Debug: Print tokenized inputs
print(f"Tokenized Input IDs: {input_ids}")
print(f"Tokenized Attention Mask: {attention_mask}")


Tokenized Input IDs: tensor([[    0, 12861,  7728,  2788,   259,     2],
        [    0, 21518,  7728,  2788,     2,     1]], device='cuda:0')
Tokenized Attention Mask: tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0]], device='cuda:0')


In [11]:
# Prepare DataLoader for validation
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
val_loader = DataLoader(val_dataset, batch_size=512, num_workers=8)


In [12]:
# Load your validation data (tokenized input_ids and attention_masks)
val_encodings, val_labels = torch.load(r'C:\Users\aless\Desktop\final project 2.1\val_encodings.pt')

# Prepare DataLoader for validation
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
val_loader = DataLoader(val_dataset, batch_size=512, num_workers=8)

# Initialize lists to store predictions and true labels
all_preds = []
all_labels = []
y_proba = []  # Store probabilities for ROC-AUC

# Set model to no_grad mode for inference
with torch.no_grad():
    for batch in val_loader:
        # Move batch to device (GPU/CPU)
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        
        # Run inference
        logits = model_custom(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get predicted classes
        predictions = torch.argmax(logits, dim=-1)
        probabilities = torch.softmax(logits, dim=-1)[:, 1]  # For ROC-AUC (probabilities for class 1)

        # Store predictions, true labels, and probabilities
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        y_proba.extend(probabilities.cpu().numpy())

C:\Users\aless\AppData\Local\Temp\ipykernel_199276\1558904231.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_encodings, val_labels = torch.load(r'C:\Users\aless\Des

In [13]:
# Check the shape and some example values
print(f"Predictions (all_preds): {all_preds[:5]}")  # Print first 5 predictions
print(f"Labels (all_labels): {all_labels[:5]}")  # Print first 5 labels


Predictions (all_preds): [1, 0, 0, 1, 0]
Labels (all_labels): [0, 0, 0, 0, 0]


In [14]:
# Print a few predictions and their corresponding labels
print("Predictions sample:", all_preds[:10])
print("True labels sample:", all_labels[:10])

# Check distribution of predictions (if the model is predicting only one class)
unique_preds, counts_preds = np.unique(all_preds, return_counts=True)
pred_class_distribution = dict(zip(unique_preds, counts_preds))
print(f"Predicted Class Distribution: {pred_class_distribution}")


Predictions sample: [1, 0, 0, 1, 0, 1, 1, 1, 0, 0]
True labels sample: [0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
Predicted Class Distribution: {0: 240790, 1: 228933}


In [15]:
# Convert lists to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
y_proba = np.array(y_proba)


In [16]:
# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)
roc_auc = roc_auc_score(all_labels, y_proba)


In [17]:
# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)
print(f"ROC-AUC Score: {roc_auc:.4f}")

Accuracy: 0.8732
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87    235207
           1       0.88      0.86      0.87    234516

    accuracy                           0.87    469723
   macro avg       0.87      0.87      0.87    469723
weighted avg       0.87      0.87      0.87    469723

Confusion Matrix:
 [[208229  26978]
 [ 32561 201955]]
ROC-AUC Score: 0.9470
